In [2]:
%load_ext watermark
%watermark -a Chan -d -p numpy,pandas,konlpy,torch,keras

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark


Using TensorFlow backend.


Chan 2019-08-20 

numpy 1.16.4
pandas 0.24.2
konlpy 0.5.1
torch 1.0.1
keras 2.2.4


In [7]:
import pandas as pd
import numpy as np
import konlpy
from utils import morp_preprocessing

# Load datasets 

In [6]:
datasets = pd.read_csv('../../data/train.csv', encoding='utf-16')
datasets.head()

,Unnamed: 0,comment,url_id,label
0,0,ㅜㅜ,77,0
1,1,ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ,17,0
2,2,헐,52,0
3,3,제시,75,0
4,4,이거인 듯,18,0


In [8]:
processed = morp_preprocessing.chat_to_morp(datasets.comment)

C:\Users\skarn\Anaconda3\envs\DataAnalysis\lib\site-packages\jpype\_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)
100%|███████████████████████████████████████████████████████████████████████████| 42000/42000 [01:15<00:00, 559.36it/s]


In [16]:
processed.shape

(42000, 10)

In [50]:
#Please create your own dataloader for new datasets of the following type

import torch
import numpy as np
from keras.preprocessing.sequence import pad_sequences
import torch.utils.data as data_utils
from sklearn.model_selection import train_test_split
import pickle
 
def load_data_set(max_len,vocab_size,batch_size):
    """
        Loads the dataset. Keras Imdb dataset for binary classifcation. Keras reuters dataset for multiclass classification
 
        Args:
            type   : {bool} 0 for binary classification returns imdb dataset. 1 for multiclass classfication return reuters set
            max_len: {int} timesteps used for padding
			vocab_size: {int} size of the vocabulary
			batch_size: batch_size
        Returns:
            train_loader: {torch.Dataloader} train dataloader
            x_test_pad  : padded tokenized test_data for cross validating
			y_test      : y_test
            word_to_id  : {dict} words mapped to indices
 
      
        """
   
    INDEX_FROM=2
    
    NUM_WORDS=vocab_size # only use top 1000 words
       # word index offset

    x_train, x_test, y_train, y_test = train_test_split(processed, np.asarray(datasets.label))
    

    with open('vocab/vocab_index.pickle', 'rb') as f:
        vocab_index = pickle.load(f)

    word_to_id =  vocab_index
    word_to_id = {k:(v+INDEX_FROM) for k,v in word_to_id.items()}
#         word_to_id["<PAD>"] = 0
#         word_to_id["<START>"] = 1
#         word_to_id["<UNK>"] = 2

    id_to_word = {value:key for key,value in word_to_id.items()}
#     x = np.concatenate([x_train, x_test])
#     y = np.concatenate([y_train, y_test])
#     n_train = x.shape[0] - 1000
#     n_valid = 1000

#     x_train = x[:n_train]
#     y_train = y[:n_train]
#     x_test = x[n_train:n_train+n_valid]
#     y_test = y[n_train:n_train+n_valid]


    #embeddings = load_glove_embeddings("../../GloVe/glove.6B.50d.txt",word_to_id,50)
    x_train_pad = pad_sequences(x_train,maxlen=max_len)
    x_test_pad = pad_sequences(x_test,maxlen=max_len)


    train_data = data_utils.TensorDataset(torch.from_numpy(x_train_pad).type(torch.LongTensor),torch.from_numpy(y_train).type(torch.DoubleTensor))
    train_loader = data_utils.DataLoader(train_data,batch_size=batch_size,drop_last=True)
    return train_loader,x_test_pad,y_test,word_to_id


# Model

In [18]:
import torch,keras
import numpy as np
from torch.autograd import Variable
import torch.nn.functional as F
import torch.utils.data as data_utils
 
class StructuredSelfAttention(torch.nn.Module):
    """
    The class is an implementation of the paper A Structured Self-Attentive Sentence Embedding including regularization
    and without pruning. Slight modifications have been done for speedup
    """
   
    def __init__(self,batch_size,lstm_hid_dim,d_a,r,max_len,emb_dim=100,vocab_size=None,use_pretrained_embeddings = False,embeddings=None,type=0,n_classes = 1):
        """
        Initializes parameters suggested in paper
 
        Args:
            batch_size  : {int} batch_size used for training
            lstm_hid_dim: {int} hidden dimension for lstm
            d_a         : {int} hidden dimension for the dense layer
            r           : {int} attention-hops or attention heads
            max_len     : {int} number of lstm timesteps
            emb_dim     : {int} embeddings dimension
            vocab_size  : {int} size of the vocabulary
            use_pretrained_embeddings: {bool} use or train your own embeddings
            embeddings  : {torch.FloatTensor} loaded pretrained embeddings
            type        : [0,1] 0-->binary_classification 1-->multiclass classification
            n_classes   : {int} number of classes
 
        Returns:
            self
 
        Raises:
            Exception
        """
        super(StructuredSelfAttention,self).__init__()
       
        self.embeddings,emb_dim = self._load_embeddings(use_pretrained_embeddings,embeddings,vocab_size,emb_dim)
        self.lstm = torch.nn.LSTM(emb_dim,lstm_hid_dim,1,batch_first=True)
        self.linear_first = torch.nn.Linear(lstm_hid_dim,d_a)
        self.linear_first.bias.data.fill_(0)
        self.linear_second = torch.nn.Linear(d_a,r)
        self.linear_second.bias.data.fill_(0)
        self.n_classes = n_classes
        self.linear_final = torch.nn.Linear(lstm_hid_dim,self.n_classes)
        self.batch_size = batch_size       
        self.max_len = max_len
        self.lstm_hid_dim = lstm_hid_dim
        self.hidden_state = self.init_hidden()
        self.r = r
        self.type = type
                 
    def _load_embeddings(self,use_pretrained_embeddings,embeddings,vocab_size,emb_dim):
        """Load the embeddings based on flag"""
       
        if use_pretrained_embeddings is True and embeddings is None:
            raise Exception("Send a pretrained word embedding as an argument")
           
        if not use_pretrained_embeddings and vocab_size is None:
            raise Exception("Vocab size cannot be empty")
   
        if not use_pretrained_embeddings:
            word_embeddings = torch.nn.Embedding(vocab_size,emb_dim,padding_idx=0)
            
        elif use_pretrained_embeddings:
            word_embeddings = torch.nn.Embedding(embeddings.size(0), embeddings.size(1))
            word_embeddings.weight = torch.nn.Parameter(embeddings)
            emb_dim = embeddings.size(1)
            
        return word_embeddings,emb_dim
       
        
    def softmax(self,input, axis=1):
        """
        Softmax applied to axis=n
 
        Args:
           input: {Tensor,Variable} input on which softmax is to be applied
           axis : {int} axis on which softmax is to be applied
 
        Returns:
            softmaxed tensors
 
       
        """
 
        input_size = input.size()
        trans_input = input.transpose(axis, len(input_size)-1)
        trans_size = trans_input.size()
        input_2d = trans_input.contiguous().view(-1, trans_size[-1])
        soft_max_2d = F.softmax(input_2d)
        soft_max_nd = soft_max_2d.view(*trans_size)
        return soft_max_nd.transpose(axis, len(input_size)-1)
       
        
    def init_hidden(self):
        return (Variable(torch.zeros(1,self.batch_size,self.lstm_hid_dim)),Variable(torch.zeros(1,self.batch_size,self.lstm_hid_dim)))
       
        
    def forward(self,x):
        embeddings = self.embeddings(x)       
        outputs, self.hidden_state = self.lstm(embeddings.view(self.batch_size,self.max_len,-1),self.hidden_state)       
        x = F.tanh(self.linear_first(outputs))       
        x = self.linear_second(x)       
        x = self.softmax(x,1)       
        attention = x.transpose(1,2)       
        sentence_embeddings = attention@outputs       
        avg_sentence_embeddings = torch.sum(sentence_embeddings,1)/self.r
       
        if not bool(self.type):
            output = F.sigmoid(self.linear_final(avg_sentence_embeddings))
           
            return output,attention
        else:
            return F.log_softmax(self.linear_final(avg_sentence_embeddings)),attention
       

    #Regularization
    def l2_matrix_norm(self,m):
        """
        Frobenius norm calculation
 
        Args:
           m: {Variable} ||AAT - I||
 
        Returns:
            regularized value
 
       
        """
        return torch.sum(torch.sum(torch.sum(m**2,1),1)**0.5).type(torch.DoubleTensor)

In [60]:
train_loader, x_test_pad, y_test, word_to_id = load_data_set(200, 30000, 512)  # loading imdb dataset
attention_model = StructuredSelfAttention(batch_size=train_loader.batch_size,
                                              lstm_hid_dim=50,
                                              d_a=100,
                                              r=10,
                                              vocab_size=30000, 
                                              max_len=200, 
                                              type=0, 
                                              n_classes=1,
                                              use_pretrained_embeddings=False,
                                              embeddings=None)

# Train

In [21]:
import torch
from torch.autograd import Variable
 
def train(attention_model,train_loader,criterion,optimizer,epochs = 5,use_regularization = False,C=0,clip=False):
    """
        Training code
 
        Args:
            attention_model : {object} model
            train_loader    : {DataLoader} training data loaded into a dataloader
            optimizer       :  optimizer
            criterion       :  loss function. Must be BCELoss for binary_classification and NLLLoss for multiclass
            epochs          : {int} number of epochs
            use_regularizer : {bool} use penalization or not
            C               : {int} penalization coeff
            clip            : {bool} use gradient clipping or not
       
        Returns:
            accuracy and losses of the model
 
      
        """
    losses = []
    accuracy = []
    for i in range(epochs):
        print("Running EPOCH",i+1)
        total_loss = 0
        n_batches = 0
        correct = 0
       
        for batch_idx,train in enumerate(train_loader):
 
            attention_model.hidden_state = attention_model.init_hidden()
            x,y = Variable(train[0]),Variable(train[1])
            y_pred,att = attention_model(x)
           
            #penalization AAT - I
            if use_regularization:
                attT = att.transpose(1,2)
                identity = torch.eye(att.size(1))
                identity = Variable(identity.unsqueeze(0).expand(train_loader.batch_size,att.size(1),att.size(1)))
                penal = attention_model.l2_matrix_norm(att@attT - identity)
           
            
            if not bool(attention_model.type) :
                #binary classification
                #Adding a very small value to prevent BCELoss from outputting NaN's
                correct+=torch.eq(torch.round(y_pred.type(torch.DoubleTensor).squeeze(1)),y).data.sum()
                if use_regularization:
                    try:
                        loss = criterion(y_pred.type(torch.DoubleTensor).squeeze(1)+1e-8,y) + C * penal/train_loader.batch_size
                       
                    except RuntimeError:
                        raise Exception("BCELoss gets nan values on regularization. Either remove regularization or add very small values")
                else:
                    loss = criterion(y_pred.type(torch.DoubleTensor).squeeze(1),y)
                
            
            else:
                
                correct+=torch.eq(torch.max(y_pred,1)[1],y.type(torch.LongTensor)).data.sum()
                if use_regularization:
                    loss = criterion(y_pred,y) + (C * penal/train_loader.batch_size).type(torch.FloatTensor)
                else:
                    loss = criterion(y_pred,y)
               
 
            total_loss+=loss.data
            optimizer.zero_grad()
            loss.backward()
           
            #gradient clipping
            if clip:
                torch.nn.utils.clip_grad_norm(attention_model.parameters(),0.5)
            optimizer.step()
            n_batches+=1
           
        print("avg_loss is",total_loss/n_batches)
        print("Accuracy of the model",correct/(n_batches*train_loader.batch_size))
        losses.append(total_loss/n_batches)
        accuracy.append(correct/(n_batches*train_loader.batch_size))
    return losses,accuracy
 
def evaluate(attention_model,x_test,y_test):
    """
        cv results
 
        Args:
            attention_model : {object} model
            x_test          : {nplist} x_test
            y_test          : {nplist} y_test
       
        Returns:
            cv-accuracy
 
      
    """
   
    attention_model.batch_size = x_test.shape[0]
    attention_model.hidden_state = attention_model.init_hidden()
    x_test_var = Variable(torch.from_numpy(x_test).type(torch.LongTensor))
    y_test_pred,_ = attention_model(x_test_var)
    if bool(attention_model.type):
        y_preds = torch.max(y_test_pred,1)[1]
        y_test_var = Variable(torch.from_numpy(y_test).type(torch.LongTensor))
       
    else:
        y_preds = torch.round(y_test_pred.type(torch.DoubleTensor).squeeze(1))
        y_test_var = Variable(torch.from_numpy(y_test).type(torch.DoubleTensor))
       
    return torch.eq(y_preds,y_test_var).data.sum()/x_test_var.size(0)
 
def get_activation_wts(attention_model,x):
    """
        Get r attention heads
 
        Args:
            attention_model : {object} model
            x               : {torch.Variable} input whose weights we want
       
        Returns:
            r different attention weights
 
      
    """
    attention_model.batch_size = x.size(0)
    attention_model.hidden_state = attention_model.init_hidden()
    _,wts = attention_model(x)
    return wts

In [61]:
loss = torch.nn.BCELoss()
optimizer = torch.optim.RMSprop(attention_model.parameters())
losses, accuracy = train(attention_model, train_loader, loss, optimizer, epochs=5,
                         use_regularization=True, C=0.03, clip=True)

Running EPOCH 1


C:\Users\skarn\Anaconda3\envs\DataAnalysis\lib\site-packages\torch\nn\functional.py:1320: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
C:\Users\skarn\Anaconda3\envs\DataAnalysis\lib\site-packages\ipykernel_launcher.py:91: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
C:\Users\skarn\Anaconda3\envs\DataAnalysis\lib\site-packages\torch\nn\functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
C:\Users\skarn\Anaconda3\envs\DataAnalysis\lib\site-packages\ipykernel_launcher.py:74: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.


avg_loss is tensor(0.2898, dtype=torch.float64)
Accuracy of the model tensor(0)
Running EPOCH 2
avg_loss is tensor(0.2387, dtype=torch.float64)
Accuracy of the model tensor(0)
Running EPOCH 3
avg_loss is tensor(0.2048, dtype=torch.float64)
Accuracy of the model tensor(0)
Running EPOCH 4
avg_loss is tensor(0.1801, dtype=torch.float64)
Accuracy of the model tensor(0)
Running EPOCH 5


RuntimeError: $ Torch: not enough memory: you tried to allocate 0GB. Buy new RAM! at ..\aten\src\TH\THGeneral.cpp:201

In [ ]:
import vi

In [54]:
test_last_idx = 100
wts = get_activation_wts(attention_model,
                         Variable(torch.from_numpy(x_test_pad[:test_last_idx]).type(torch.LongTensor)))
print(wts.size())
visualize_attention(wts, x_test_pad[:test_last_idx], word_to_id, filename='attention.html')

C:\Users\skarn\Anaconda3\envs\DataAnalysis\lib\site-packages\ipykernel_launcher.py:91: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Attention weights for the testing data in binary classification are: tensor([[[6.6596e-01, 3.3026e-01, 3.7529e-03,  ..., 3.5268e-10,
          3.3828e-10, 2.4966e-07],
         [2.1786e-09, 1.3291e-09, 2.9249e-09,  ..., 6.9241e-12,
          4.8197e-12, 3.0312e-01],
         [1.2909e-08, 1.2504e-09, 9.4795e-09,  ..., 3.5037e-05,
          1.9510e-04, 4.9763e-06],
         ...,
         [1.5439e-11, 1.2078e-12, 6.0075e-12,  ..., 1.5603e-05,
          1.5612e-06, 8.1863e-04],
         [1.5809e-04, 4.5176e-03, 1.4398e-01,  ..., 7.2862e-15,
          4.4046e-13, 3.7180e-13],
         [4.8935e-08, 5.1401e-09, 4.4072e-08,  ..., 9.3209e-03,
          9.5218e-03, 1.0529e-03]],

        [[6.6596e-01, 3.3026e-01, 3.7529e-03,  ..., 9.7322e-09,
          1.5156e-08, 1.2897e-08],
         [8.6980e-10, 5.3064e-10, 1.1678e-09,  ..., 9.6619e-09,
          3.5849e-08, 1.7947e-08],
         [1.3004e-08, 1.2595e-09, 9.5490e-09,  ..., 2.3950e-07,
          9.9275e-08, 8.4737e-08],
         ...,
         [

In [58]:
evaluate(attention_model, x_test, y_test)

<function keras_preprocessing.sequence.pad_sequences(sequences, maxlen=None, dtype='int32', padding='pre', truncating='pre', value=0.0)>